In [1]:
import numpy as np

import pickle as pic

In [3]:
from utils import *
from templates import *
from signal_strength import SIGNAL_STRENGTHS_METHODS
from data_loader import EntireTraceIterator

In [ ]:
traces_path = "..\\acquisition\\1000000_maskshuffle_allrounds\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Masking-Shuffling.mat"
key_path = "..\\acquisition\\1000000_maskshuffle_allrounds\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Masking-Shuffling.log"

In [ ]:
NUM_TRACES = 1_000_000
TRACE_SIZE = 200_000
PER_DIVISION = 250_000
data_loader = EntireTraceIterator(traces_path, key_path, nr_populations=1, nr_scenarios=1, trace_size=TRACE_SIZE, traces_per_division=PER_DIVISION, parse_output="keyshares+perms")

# Saving the seeds, key and labels to a single file

In [ ]:
for seeds, key, _ in data_loader((0,), (0,), return_traces=False):
    seeds = seeds[0][0]
    key = np.array(key[0])
    break

In [ ]:
rws_perms_labels = np.zeros(NUM_TRACES, dtype=int)
rws_masks_labels = np.zeros((KEYROUND_WIDTH_B4, NR_SHARES, NUM_TRACES), dtype=int)
round_perms_labels = np.zeros(NUM_TRACES, dtype=int)
copy_perms_labels = np.zeros((LATEST_ROUND - EARLIEST_ROUND, NUM_TRACES), dtype=int)
round_masks_labels = np.zeros((LATEST_ROUND - EARLIEST_ROUND, BLOCK_WIDTH_B4, NR_SHARES, NUM_TRACES), dtype=int)

# Iterate through the dataset iteratively because it does not fit in RAM
for j, (seeds_sub, key, output_sub) in enumerate(data_loader((0,), (0,), return_traces=False)):
    rws_perms, rws_masks, round_perms, copy_perms, round_masks = get_all_labels(seeds_sub[0][0], key[0], output_sub[0][0])
    rws_perms_labels[..., j * PER_DIVISION:j * PER_DIVISION + rws_perms.shape[-1]] = rws_perms
    rws_masks_labels[..., j * PER_DIVISION:j * PER_DIVISION + rws_masks.shape[-1]] = rws_masks
    round_perms_labels[..., j * PER_DIVISION:j * PER_DIVISION + round_perms.shape[-1]] = round_perms
    copy_perms_labels[..., j * PER_DIVISION:j * PER_DIVISION + copy_perms.shape[-1]] = copy_perms
    round_masks_labels[..., j * PER_DIVISION:j * PER_DIVISION + round_masks.shape[-1]] = round_masks

In [ ]:
data_loader.empty_traces

[324360]

In [ ]:
rws_perms_labels = np.concatenate((rws_perms_labels[..., :499999], rws_perms_labels[..., 500000:]), axis=-1)
rws_masks_labels = np.concatenate((rws_masks_labels[..., :499999], rws_masks_labels[..., 500000:]), axis=-1)
round_perms_labels = np.concatenate((round_perms_labels[..., :499999], round_perms_labels[..., 500000:]), axis=-1)
copy_perms_labels = np.concatenate((copy_perms_labels[..., :499999], copy_perms_labels[..., 500000:]), axis=-1)
round_masks_labels = np.concatenate((round_masks_labels[..., :499999], round_masks_labels[..., 500000:]), axis=-1)

In [ ]:
with open("labels_pics/seeds_key_labels_1000000.pic", "wb") as w:
    pic.dump((seeds, key, rws_perms_labels, round_perms_labels, copy_perms_labels, rws_masks_labels, round_masks_labels), w)

In [4]:
trainval, test = train_test_split(np.arange(seeds.shape[0]), train_size=750_000, random_state=0)
rws_perms_labels_trainval, rws_perms_labels_test = rws_perms_labels[..., trainval], rws_perms_labels[..., test]
round_perms_labels_trainval, round_perms_labels_test = round_perms_labels[..., trainval], round_perms_labels[..., test]
copy_perms_labels_trainval, copy_perms_labels_test = copy_perms_labels[..., trainval], copy_perms_labels[..., test]
rws_masks_labels_trainval, rws_masks_labels_test = rws_masks_labels[..., trainval], rws_masks_labels[..., test]
round_masks_labels_trainval, round_masks_labels_test = round_masks_labels[..., trainval], round_masks_labels[..., test]

In [5]:
TRAIN_SIZE = int(0.9 * 750_000)

# Templates

## RWS permutations

### Profiling: cross-validation

In [6]:
n_folds = 5
num_features = [750, 1_000, 1_500]

with open("traces_and_splits/traces_rws_only.pic", "rb") as r:
    traces_trainval = pic.load(r)
    traces_trainval = traces_trainval[trainval, 150:20000]

signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"]("leakage_points_smaller_windows/f_rws_perms_sost.pic")
best_model, best_feat, best_result = b4_gridsearch_cv(signal_strength, n_folds, TRAIN_SIZE, num_features, traces_trainval, rws_perms_labels_trainval)
with open(f"saved_templates/rws_perms_{num_features[best_feat]}_features.pic", "wb") as w:
    pic.dump(best_model, w)
print(f"Accuracy: {best_result}")

SOST [num features: 750]: 0.6640 ± 0.001632 ([0.66268    0.66466667 0.66646667 0.66177333 0.66429333]).
SOST [num features: 1000]: 0.6996 ± 0.001135 ([0.69830667 0.69936    0.70124    0.6986     0.70061333]).
SOST [num features: 1500]: 0.6533 ± 0.0008565 ([0.65357333 0.65294667 0.65381333 0.65190667 0.65442667]).
-- BEST NUM_FEATURES: 1000--
Accuracy: 0.699624


### Performance on extraction

In [48]:
rws_perms_probas = np.zeros(rws_perms_labels_test.shape + (KEYROUND_WIDTH_B4,))

with open("traces_and_splits/traces_rws_only.pic", "rb") as r:
    traces_test = pic.load(r)
    traces_test = traces_test[test, 150:20000]

sig_strength = SIGNAL_STRENGTHS_METHODS["SOST"]("leakage_points_smaller_windows/f_rws_perms_sost.pic")
sig_strength.num_features = 1000
with open(f"saved_templates/rws_perms_{1000}_features.pic", "rb") as r:
    best_model = pic.load(r)

rws_perms_probas = best_model.predict_proba(sig_strength.transform(traces_test))
accuracy = np.count_nonzero(np.argmax(rws_perms_probas, axis=1) == rws_perms_labels_test) / rws_perms_labels_test.shape[0]
print(f"Test accuracy: {accuracy}")

Test accuracy: 0.6996187984751939


## RWS masks

### Profiling: cross-validation

In [7]:
n_folds = 5
num_features = [20]

with open("traces_and_splits/traces_rws_only.pic", "rb") as r:
    traces_trainval = pic.load(r)
    traces_trainval = traces_trainval[trainval, 200:20060]

best_results = [] 
for i in range(7):
    traces_trainval_sub = traces_trainval[:, i * (traces_trainval.shape[1] // 7):(i+1) * (traces_trainval.shape[1] // 7)]

    for keyround_idx in range(i * (98 // 7), (i+1) * (98 // 7)):
        for share_idx in range(NR_SHARES):
            signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"](f"leakage_points_smaller_windows/f_rws_masks_{keyround_idx}_{share_idx}_sost.pic")
            best_model, best_feat, best_result = b4_gridsearch_cv(signal_strength, n_folds, TRAIN_SIZE, num_features, traces_trainval_sub, rws_masks_labels_trainval[keyround_idx, share_idx])
            best_results.append(best_result)
            with open(f"saved_templates/rws_masks_{keyround_idx}_{share_idx}_{num_features[best_feat]}_features.pic", "wb") as w:
                pic.dump(best_model, w)
print(f"Accuracy: {np.mean(best_results)} +/- {np.std(best_results)}")

SOST [num features: 20]: 0.07775 ± 0.0007111 ([0.07861333 0.07730667 0.07742667 0.07685333 0.07857333]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07721 ± 0.0007060 ([0.07597333 0.07804    0.07701333 0.07737333 0.07766667]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07359 ± 0.0008896 ([0.07365333 0.07386667 0.07478667 0.07361333 0.07202667]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07715 ± 0.0004901 ([0.07762667 0.07766667 0.07666667 0.07648    0.07730667]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07687 ± 0.0004569 ([0.07742667 0.07614667 0.0768     0.07729333 0.07670667]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07750 ± 0.001191 ([0.07833333 0.07741333 0.07857333 0.07793333 0.07525333]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07792 ± 0.001458 ([0.07969333 0.07736    0.07924    0.0756     0.07769333]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07823 ± 0.0008318 ([0.07884    0.07714667 0.0794   

### Performance on extraction

In [49]:
rws_masks_probas = np.zeros(rws_masks_labels_test.shape + (len(KEY_ALPHABET),))

with open("traces_and_splits/traces_rws_only.pic", "rb") as r:
    traces_test = pic.load(r)
    traces_test = traces_test[test, 200:20060]

accuracies = [] 
for i in range(7):
    traces_test_sub = traces_test[:, i * (traces_test.shape[1] // 7):(i+1) * (traces_test.shape[1] // 7)]

    for keyround_idx in range(i * (98 // 7), (i+1) * (98 // 7)):
        for share_idx in range(NR_SHARES):
            sig_strength = SIGNAL_STRENGTHS_METHODS["SOST"](f"leakage_points_smaller_windows/f_rws_masks_{keyround_idx}_{share_idx}_sost.pic")
            sig_strength.num_features = 20
            with open(f"saved_templates/rws_masks_{keyround_idx}_{share_idx}_{20}_features.pic", "rb") as r:
                best_model = pic.load(r)

            rws_masks_probas[keyround_idx, share_idx] = best_model.predict_proba(sig_strength.transform(traces_test_sub))
            accuracy = np.count_nonzero(np.argmax(rws_masks_probas[keyround_idx, share_idx], axis=1) == rws_masks_labels_test[keyround_idx, share_idx]) / rws_masks_labels_test[keyround_idx, share_idx].shape[0]
            accuracies.append(accuracy)
print(f"Test accuracy: {np.mean(accuracies)} +/- {np.std(accuracies)}")

Test accuracy: 0.07712973709037693 +/- 0.0018317229050269575


## Round permutations

### Profiling: cross-validation

In [19]:
n_folds = 5
num_features = [150, 200, 250]

with open("traces_and_splits/traces_round_perm_only.pic", "rb") as r:
    traces_trainval = pic.load(r)
    traces_trainval = traces_trainval[trainval]

signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"]("leakage_points_smaller_windows/f_round_perms_sost.pic")
best_model, best_feat, best_result = b4_gridsearch_cv(signal_strength, n_folds, TRAIN_SIZE, num_features, traces_trainval, round_perms_labels_trainval)
with open(f"saved_templates/round_perms_{num_features[best_feat]}_features.pic", "wb") as w:
    pic.dump(best_model, w)
print(f"Accuracy: {np.mean(best_result)}")

SOST [num features: 150]: 0.6543 ± 0.001900 ([0.65481333 0.65101333 0.65629333 0.6534     0.65578667]).
SOST [num features: 200]: 0.6604 ± 0.001440 ([0.66045333 0.65794667 0.66242667 0.66053333 0.66088   ]).
SOST [num features: 250]: 0.6599 ± 0.001379 ([0.659      0.65764    0.66128    0.66058667 0.661     ]).
-- BEST NUM_FEATURES: 200--
Accuracy: 0.660448


### Performance on extraction

In [50]:
round_perms_probas = np.zeros(round_perms_labels_test.shape + (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4,))

with open("traces_and_splits/traces_round_perm_only.pic", "rb") as r:
    traces_test = pic.load(r)
    traces_test = traces_test[test]

sig_strength = SIGNAL_STRENGTHS_METHODS["SOST"]("leakage_points_smaller_windows/f_round_perms_sost.pic")
sig_strength.num_features = 200
with open(f"saved_templates/round_perms_{200}_features.pic", "rb") as r:
    best_model = pic.load(r)

round_perms_probas = best_model.predict_proba(sig_strength.transform(traces_test))
accuracy = np.count_nonzero(np.argmax(round_perms_probas, axis=1) == round_perms_labels_test) / round_perms_labels_test.shape[0]
print(f"Test accuracy: {accuracy}")

Test accuracy: 0.6591586366345465


## Copy permutations and round masks

### Profiling: cross-validation

In [25]:
n_folds = 5

best_results_copy = []
best_results_masks = []
for round_idx in range(EARLIEST_ROUND, LATEST_ROUND):
    with open(f"traces_and_splits/traces_round_{round_idx}_only.pic", "rb") as r:
        traces_trainval = pic.load(r)
        traces_trainval = traces_trainval[trainval]

    signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"](f"leakage_points_smaller_windows/f_copy_perms_{round_idx}_sost.pic")
    num_features = [200]
    best_model, best_feat, best_result = b4_gridsearch_cv(signal_strength, n_folds, TRAIN_SIZE, num_features, traces_trainval, copy_perms_labels_trainval[round_idx])
    best_results_copy.append(best_result)
    with open(f"saved_templates/copy_perms_{round_idx}_{num_features[best_feat]}_features.pic", "wb") as w:
        pic.dump(best_model, w)

    for block_idx in range(BLOCK_WIDTH_B4):
        for share_idx in range(NR_SHARES):
            signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"](f"leakage_points_smaller_windows/f_round_masks_{round_idx}_{block_idx}_{share_idx}_sost.pic")
            num_features = [20]
            best_model, best_feat, best_result = b4_gridsearch_cv(signal_strength, n_folds, TRAIN_SIZE, num_features, traces_trainval, round_masks_labels_trainval[round_idx, block_idx, share_idx])
            best_results_masks.append(best_result)
            with open(f"saved_templates/round_masks_{round_idx}_{block_idx}_{share_idx}_{num_features[best_feat]}_features.pic", "wb") as w:
                pic.dump(best_model, w)
print(f"Block perm accuracy: {np.mean(best_results_copy)} +/- {np.std(best_results_copy)}")
print(f"Round masks accuracy: {np.mean(best_results_masks)} +/- {np.std(best_results_masks)}")

SOST [num features: 200]: 0.5773 ± 0.001756 ([0.57486667 0.57992    0.57833333 0.5772     0.57605333]).
-- BEST NUM_FEATURES: 200--
SOST [num features: 20]: 0.07341 ± 0.0009167 ([0.07224    0.07334667 0.0746     0.07428    0.0726    ]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07884 ± 0.0006732 ([0.07849333 0.07912    0.08004    0.07826667 0.07829333]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07198 ± 0.0005746 ([0.07129333 0.07228    0.07262667 0.07242667 0.07129333]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07519 ± 0.0007653 ([0.07377333 0.07585333 0.07533333 0.07586667 0.07513333]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07104 ± 0.0005625 ([0.0706     0.0716     0.07145333 0.07016    0.0714    ]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07365 ± 0.0008426 ([0.07208    0.07362667 0.07457333 0.07386667 0.07408   ]).
-- BEST NUM_FEATURES: 20--
SOST [num features: 20]: 0.07196 ± 0.0008456 ([0.07309333 0.07049333 0.07184

### Performance on extraction

In [52]:
copy_perms_probas = np.zeros(copy_perms_labels_test.shape + (BLOCK_WIDTH_B4,))
round_masks_probas = np.zeros(round_masks_labels_test.shape + (len(KEY_ALPHABET),))

accuracies_copy = []
accuracies_masks = []
for round_idx in range(EARLIEST_ROUND, LATEST_ROUND):
    with open(f"traces_and_splits/traces_round_{round_idx}_only.pic", "rb") as r:
        traces_test = pic.load(r)
        traces_test = traces_test[test]

    sig_strength = SIGNAL_STRENGTHS_METHODS["SOST"](f"leakage_points_smaller_windows/f_copy_perms_{round_idx}_sost.pic")
    sig_strength.num_features = 200
    with open(f"saved_templates/copy_perms_{round_idx}_{200}_features.pic", "rb") as r:
        best_model = pic.load(r)
    copy_perms_probas[round_idx] = best_model.predict_proba(sig_strength.transform(traces_test))
    accuracy = np.count_nonzero(np.argmax(copy_perms_probas[round_idx], axis=1) == copy_perms_labels_test[round_idx]) / copy_perms_labels_test[round_idx].shape[0]
    accuracies_copy.append(accuracy)

    for block_idx in range(BLOCK_WIDTH_B4):
        for share_idx in range(NR_SHARES):
            sig_strength = SIGNAL_STRENGTHS_METHODS["SOST"](f"leakage_points_smaller_windows/f_round_masks_{round_idx}_{block_idx}_{share_idx}_sost.pic")
            sig_strength.num_features = 20
            with open(f"saved_templates/round_masks_{round_idx}_{block_idx}_{share_idx}_{20}_features.pic", "rb") as r:
                best_model = pic.load(r)
            round_masks_probas[round_idx, block_idx, share_idx] = best_model.predict_proba(sig_strength.transform(traces_test))
            accuracy = np.count_nonzero(np.argmax(round_masks_probas[round_idx, block_idx, share_idx], axis=1) == round_masks_labels_test[round_idx, block_idx, share_idx]) / round_masks_labels_test[round_idx, block_idx, share_idx].shape[0]
            accuracies_masks.append(accuracy)

print(f"Block perm test accuracy: {np.mean(accuracies_copy)} +/- {np.std(accuracies_copy)}")
print(f"Round masks test accuracy: {np.mean(accuracies_masks)} +/- {np.std(accuracies_masks)}")

Block perm test accuracy: 0.580112891880139 +/- 0.0021345482247458114
Round masks test accuracy: 0.07373960108085331 +/- 0.0020994398874153645


## Combining the masks and RSIs probabilities to recover the entire key

In [59]:
# Put dimension of observations first
rws_perms_probas = np.moveaxis(rws_perms_probas, -2, 0)
rws_masks_probas = np.moveaxis(rws_masks_probas, -2, 0)
round_perms_probas = np.moveaxis(round_perms_probas, -2, 0)
copy_perms_probas = np.moveaxis(copy_perms_probas, -2, 0)
round_masks_probas = np.moveaxis(round_masks_probas, -2, 0)

In [61]:
per_trace_rws = rws_classifications_per_trace("preds_per_traces/per_trace_750000_orig_rws_templates.pic", seeds[test], rws_perms_probas, rws_masks_probas, parallel=True)

per_trace_rounds = classifications_per_trace("preds_per_traces/per_trace_750000_orig_round_templates.pic", seeds[test], round_perms_probas, copy_perms_probas, round_masks_probas, parallel=True)

per_trace_total = per_trace_rws + per_trace_rounds
recovered_key = reconstruct_key(per_trace_total)

In [62]:
np.count_nonzero(reconstruct_key(per_trace_total) == key) / 512

0.20703125